# Section 3.1 Single Model Diagnostics
* How can we determine that our MCMC model has converged?*

In [ ]:
import arviz as az

## Learning Objectives
* Understand Hamiltonian Monte Carlo sampler basics
* Visualize and interpret single model diagnostics such as TracePlot, Autocorrelation, and Energy Plot
* Calculate and interpret single model diagnostic calculations 
* Gain profiency with Xarray, NetCDF, and az.InferenceData objects

## MCMC is great but its not perfect
Recall that Monte Carlo Markov Chain are handy mathematical trick that let's us compute posteriors, without needing to compute the denonimator of Bayes rule, but it has tradeoffs. Most notably **finite Monte Carlo Markov Chains are not gauranteed to converge to the true parameter value**

This is not to say finite MCMC chains are always wrong, but it does mean that when using MCMC inference methods it is prudent to check the results diagnostic methods. In this tutorial we'll cover both numerical and visual methods

## Numerical Diagnostics
Numerical diagnostics are great for quickly understanding the characteristics of an MCMC chain. We will be covering two

* Effective Sample Size
* Rhat